# Multi-Agent System (AAI-520 Final Project)
---

Team Members (Group 4)
- Atul Aneja
- Juan Triana Martinez
- Mostafa Zamaniturk
- Ramesh Dhanasekaran
- Victoria Dorn

Code can be found on [github](https://github.com/vdorn5/aai-520-final-project/tree/main).

## Investment Research Agentic AI 

This nobebook walks through the creation of an autonomous multi-agent system designed for performing dynamic, self-improving investment research using LLMs, financial data, and structured agent workflows. Built with CrewAI and LangGraph to orchestrate specialist agents in a collaborative research pipeline.

### How to run this notebook?

Clone the [github](https://github.com/vdorn5/aai-520-final-project/tree/main) repository and follow instructions on readme. The recommendation is to use the devcontainer setup, but alternatively you can install the `.devcontainer/requirements.txt` in your own setup. *Make sure to update the .env file with your OPEN_API_KEY, FRED_API_KEY, and NEWS_API_KEY.*

---
## Code Execution

In [1]:
# Environment setup and imports
import os, sys, subprocess
from pathlib import Path
from dotenv import load_dotenv

# Resolve project root to current working directory of this notebook
PROJECT_ROOT = Path.cwd()
SRC_DIR = PROJECT_ROOT / "src"

# Ensure src is importable
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

print("Project root:", PROJECT_ROOT)
print("Using src path:", SRC_DIR)

# Load .env explicitly from project root
load_dotenv(PROJECT_ROOT / ".env")

# Optional: ensure requirements are present (no-op if already installed)
REQS = PROJECT_ROOT / "requirements.txt"
if REQS.exists() and os.getenv("INSTALL_REQUIREMENTS", "0") == "1":
    print("Installing requirements (optional)...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(REQS)], check=False)

Project root: /workspaces/aai-520-final-project
Using src path: /workspaces/aai-520-final-project/src


---
### Inlined Code: `src/tools/news_client.py`

 - **Module path**: `src.tools.news_client`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [2]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
NEWS_API_KEY = os.getenv("NEWS_API_KEY")

def get_company_news(company_name):
    url = f"https://newsapi.org/v2/everything?q={company_name}&sortBy=publishedAt&apiKey={NEWS_API_KEY}"
    response = requests.get(url)
    articles = response.json().get("articles", [])[:5]
    return [article["title"] + " - " + article["description"] for article in articles if article.get("description")]


---
### Inlined Code: `src/agents/news_analyst_agent.py`

 - **Module path**: `src.agents.news_analyst_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [3]:
from crewai import Agent
from src.tools.news_client import get_company_news
from tools.news_analysis_tools import NewsAnalysisTools
# Create a News Analyst agent
news_analyst = Agent(
    role='News Analyst',
    goal='Analyze the latest news and market sentiment for a given company.',
    backstory=(
        'A specialist in processing and interpreting financial news. You can '
        'gauge market sentiment, identify key narratives, and detect significant '
        'events reported in the media using your advanced analysis pipeline.'
    ),
    tools=[NewsAnalysisTools.news_analysis_pipeline],
    verbose=True,
    allow_delegation=False
)

class NewsAnalystAgent:
    def __init__(self, llm):
        self.llm = llm
        self.agent = Agent(
            role="News Analyst",
            goal="Analyze recent news for company sentiment",
            backstory="Expert in parsing financial news and identifying relevant trends.",
            verbose=True,
            allow_delegation=False,
            tools=[],
            llm=self.llm
        )

    def analyze(self, company_name):
        news = get_company_news(company_name)
        prompt = f"Summarize the following news about {company_name}:\n\n" + "\n".join(news)
        response = self.llm.complete(prompt)
        return response


ModuleNotFoundError: No module named 'tools.news_analysis_tools'

---
### Inlined Code: `src/tools/yfinance_client.py`

 - **Module path**: `src.tools.yfinance_client`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from crewai.tools import tool
import yfinance as yf
# --- Tool for Basic Stock Data ---
@tool("Stock Ticker Data Tool")
def get_stock_data(ticker: str) -> dict:
    """A tool to get basic financial data for a given stock ticker."""
    # Implementation remains the same...
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "longName": info.get("longName"),
        "sector": info.get("sector"),
        "industry": info.get("industry"),
        "marketCap": info.get("marketCap"),
        "trailingPE": info.get("trailingPE"),
        "forwardPE": info.get("forwardPE"),
        "dividendYield": info.get("dividendYield"),
        "fiftyTwoWeekHigh": info.get("fiftyTwoWeekHigh"),
        "fiftyTwoWeekLow": info.get("fiftyTwoWeekLow"),
        "regularMarketPrice": info.get("regularMarketPrice")
    }

# --- Tool for Financial Statements (for EarningsAnalyst) ---
@tool("Company Financial Statements Tool")
def get_financial_statements(ticker: str) -> str:
    """
    A tool to get the most recent annual financial statements (Income Statement,
    Balance Sheet, and Cash Flow) for a given stock ticker.
    """
    stock = yf.Ticker(ticker)
    
    # Fetch the most recent annual data
    income_statement = stock.income_stmt.iloc[:, 0]
    balance_sheet = stock.balance_sheet.iloc[:, 0]
    cash_flow = stock.cashflow.iloc[:, 0]
    
    # Format into a single string for the LLM
    report = f"""
    Income Statement:\n{income_statement.to_string()}\n
    Balance Sheet:\n{balance_sheet.to_string()}\n
    Cash Flow Statement:\n{cash_flow.to_string()}
    """
    return report

def get_yfinance_financials(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    return financials.to_string()

def get_analyst_recommendations(ticker):
    stock = yf.Ticker(ticker)
    return stock.recommendations.tail(3).to_string()


---
### Inlined Code: `src/agents/earnings_analyst_agent.py`

 - **Module path**: `src.agents.earnings_analyst_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from crewai import Agent
from tools.yfinance_client import get_stock_data, get_financial_statements
from tools.memory_tools import read_memory
# Create an Earnings Analyst agent
earnings_analyst = Agent(
  role='Earnings Analyst',
  goal='Analyze the financial statements of a company to assess its financial health and performance, considering any past analyses.',
  backstory=(
    'A meticulous financial analyst with a deep understanding of corporate finance. '
    'You specialize in dissecting income statements, balance sheets, and cash flow statements '
    'to uncover underlying trends and financial stability. You also consult past reports to see what has changed.'
  ),
  tools=[get_stock_data, get_financial_statements, read_memory], # Give this agent the read_memory tool
  verbose=True,
  allow_delegation=False
)
from src.tools.yfinance_client import get_yfinance_financials
from src.tools.yfinance_client import get_yfinance_financials, get_analyst_recommendations
import pandas as pd
import io
import json
from functools import lru_cache

@lru_cache(maxsize=50)
def get_financials_cached(ticker):
    """Cached version of financial fetcher to avoid repeated API calls."""
    return get_yfinance_financials(ticker)

class EarningsAnalystAgent:
    def __init__(self, llm):
        self.llm = llm

        # Create the agent with the tools
        self.agent = Agent(
            role="Earnings Analyst",
            goal="Review earnings reports and interpret key financial data.",
            backstory="Expert in financial analysis and equity research.",
            verbose=True,
            llm=self.llm
        )

    # Helper: parse financials
    def _parse_financials(self, financials_str):
        """Convert the yfinance .to_string() output into a DataFrame for quick extraction."""
        try:
            df = pd.read_csv(io.StringIO(financials_str))
            return df
        except Exception as e:
            print(f"Warning: Could not parse financials as CSV: {e}")
            # fallback simple parse
            lines = [line.strip() for line in financials_str.split("\n") if line.strip()]
            return lines   

    # Main analyze method
    def analyze(self, ticker):
        print(f"\n[+] Running Earnings Analysis for {ticker}...\n")

        # Fetch data directly
        financials = get_financials_cached(ticker)
        recommendations = get_analyst_recommendations(ticker)

        # Error Handling 
        if not financials:
            return f"No financial data available for {ticker}."
        if not recommendations:
            recommendations = "No analyst recommendations available."

        # Attempt to parse numeric insights (for potential future use)
        parsed_financials = self._parse_financials(financials)
        if isinstance(parsed_financials, pd.DataFrame):
            print(f"Parsed financials into DataFrame with shape: {parsed_financials.shape}")
        else:
            print(f"Financials parsed as text with {len(parsed_financials)} lines")
        
        # Structured LLM prompt 
        prompt = f"""
You are an Earnings Analyst. 
Review the financial report and provide a structured analysis in JSON format.

**Company:** {ticker}

**Instructions:**
Analyze the financial data and respond ONLY in valid JSON format with the following structure:
{{
    "revenue": "string - Total revenue with growth percentage if available",
    "net_income": "string - Net income with growth/decline percentage",
    "eps": "string - Earnings per share with comparison to previous period",
    "growth_comment": "string - Brief analysis of growth trends and key metrics",
    "analyst_sentiment": "string - Summary of analyst recommendations and sentiment",
    "overall_sentiment": "positive|neutral|negative - Overall investment sentiment"
}}

**Financial Data to Analyze:**
1. **Quantitative extraction** - Identify key figures:
   - Total Revenue
   - Net Income  
   - Earnings Per Share (EPS)
   - Year-over-year or quarter-over-quarter changes

2. **Qualitative interpretation** - Discuss:
   - Performance trends
   - Profitability indicators
   - Growth signals

3. **Analyst sentiment** - Consider these latest recommendations:
{recommendations}

**Raw financial data:**
{financials}

Respond with ONLY the JSON object, no additional text or formatting.
"""
        response = self.llm.complete(prompt)

        # Parse JSON response
        try:
            result = json.loads(response)
        except json.JSONDecodeError as e:
            print(f"Warning: Failed to parse LLM response as JSON: {e}")
            print(f"Raw response: {response[:200]}...")
            result = {
                "error": "Failed to parse JSON response", 
                "raw_output": response,
                "revenue": "Unable to extract",
                "net_income": "Unable to extract", 
                "eps": "Unable to extract",
                "growth_comment": "Analysis failed due to parsing error",
                "analyst_sentiment": "Unable to extract",
                "overall_sentiment": "neutral"
            }

        return result


---
### Inlined Code: `src/agents/market_analyst_agent.py`

 - **Module path**: `src.agents.market_analyst_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from crewai import Agent
from tools.fred_client import get_fred_data

# Create a Market Analyst agent
market_analyst = Agent(
    role='Macroeconomic Analyst',
    goal='Provide macroeconomic context for the stock analysis.',
    backstory=(
        'An economist with expertise in tracking and interpreting broad economic indicators. '
        'Your insights help frame the company\'s performance within the larger economic picture.'
    ),
    tools=[get_fred_data],
    verbose=True,
    allow_delegation=False
)


---
### Inlined Code: `src/agents/critic_agent.py`

 - **Module path**: `src.agents.critic_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from crewai import Agent
# Create the Critic Agent
critic_agent = Agent(
    role='Quality Assurance Critic',
    goal='Critique and refine the investment report to ensure it is comprehensive, accurate, and actionable.',
    backstory=(
        'A highly experienced investment strategist with a keen eye for detail. You are tasked with '
        'reviewing financial reports to identify any gaps, logical inconsistencies, or unsupported claims. '
        'Your feedback is crucial for producing a final, high-quality investment recommendation.'
    ),
    verbose=True,
    allow_delegation=False
)

class CriticAgent:
    def __init__(self, llm):
        self.llm = llm
        self.agent = Agent(
            role="Critic",
            goal="Evaluate and provide constructive feedback on investment reports",
            backstory="A strict but fair financial critic who checks quality and completeness.",
            verbose=True,
            allow_delegation=False,
            tools=[],
            llm=self.llm
        )

    def critique(self, report_text):
        prompt = f"""
You're a financial report evaluator. Critically evaluate the following investment research report. Assess it based on:

1. Relevance to the company
2. Clarity and conciseness
3. Factual correctness
4. Completeness of the analysis

Give a score from 1 to 10 and explain your reasoning.

Report:
{report_text}
"""
        return self.llm.complete(prompt)


---
### Inlined Code: `src/utils/llm_wrapper.py`

 - **Module path**: `src.utils.llm_wrapper`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

class LLMWrapper:
    def __init__(self):
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    def complete(self, prompt):
        response = self.client.chat.completions.create(
            model="gpt-5-nano",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

    def show_models(self):
        models = self.client.models.list()

        for model in models.data:
            print(model.id)

---
### Inlined Code: `src/agents/orchestrator_agent.py`

 - **Module path**: `src.agents.orchestrator_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from src.agents.news_analyst_agent import NewsAnalystAgent
from src.agents.earnings_analyst_agent import EarningsAnalystAgent
from src.agents.market_analyst_agent import MarketAnalystAgent
from src.agents.critic_agent import CriticAgent
from src.utils.llm_wrapper import LLMWrapper

class OrchestratorAgent:
    def __init__(self):
        self.llm = LLMWrapper()
        self.news_agent = NewsAnalystAgent(self.llm)
        self.earnings_agent = EarningsAnalystAgent(self.llm)
        self.market_agent = MarketAnalystAgent(self.llm)
        self.critic_agent = CriticAgent(self.llm)

    def run(self, ticker):
        print(f"\nStarting Investment Research for: {ticker}\n")

        print("News Summary:")
        news_summary = self.news_agent.analyze(ticker)
        print(news_summary)

        print("\nEarnings Summary:")
        earnings_summary = self.earnings_agent.analyze(ticker)
        print(earnings_summary)

        print("\nMarket Context:")
        market_summary = self.market_agent.analyze()
        print(market_summary)

        # Combine all sections into one report
        full_report = f"""
    # Investment Research Report: {ticker}

    ## News Summary:
    {news_summary}

    ## Earnings Summary:
    {earnings_summary}

    ## Market Context:
    {market_summary}
    """

        print("\nRunning Critique:\n")
        critique = self.critic_agent.critique(full_report)
        print(critique)

        print("\nFinal Investment Report with Critique Generated.\n")



## Agent Modules — Sanity Check

This cell verifies that all CrewAI agent modules import correctly and exposes the expected agent instances.

In [ ]:
# Agent Modules — Sanity Check

# Using inline instead
# from agents.critic_agent import critic_agent
# from agents.earnings_analyst_agent import earnings_analyst
# from agents.market_analyst_agent import market_analyst
# from agents.news_analyst_agent import news_analyst
# from agents.investment_agents import investment_advisor

agents = [
    ("critic_agent", critic_agent),
    ("earnings_analyst", earnings_analyst),
    ("market_analyst", market_analyst),
    ("news_analyst", news_analyst),
    ("investment_advisor", investment_advisor),
]
for name, ag in agents:
    try:
        tools = getattr(ag, "tools", []) or []
        print(f"✓ {name}: role='{ag.role}', tools={len(tools)}")
    except Exception as e:
        print(f"✗ {name}: import/attr error -> {e}")# Agent Modules — Sanity Check
from agents.critic_agent import critic_agent
from agents.earnings_analyst_agent import earnings_analyst
from agents.market_analyst_agent import market_analyst
from agents.news_analyst_agent import news_analyst
from agents.investment_agents import investment_advisor

agents = [
    ("critic_agent", critic_agent),
    ("earnings_analyst", earnings_analyst),
    ("market_analyst", market_analyst),
    ("news_analyst", news_analyst),
    ("investment_advisor", investment_advisor),
]
for name, ag in agents:
    try:
        tools = getattr(ag, "tools", []) or []
        print(f"✓ {name}: role='{ag.role}', tools={len(tools)}")
    except Exception as e:
        print(f"✗ {name}: import/attr error -> {e}")

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'tools.yfinance_client'

## Tasks — Sanity Check

In [ ]:
# Tasks — Sanity Check
# Import orchestrator to ensure tasks are wired (agents and contexts assigned)
try:
    from orchestrator import run_analysis as _wired  # noqa: F401
except Exception:
    # Wiring will still be safe-checked below
    pass

from tasks.financial_tasks import (
    memory_retrieval_task,
    earnings_analysis_task,
    news_analysis_task,
    market_analysis_task,
    advisory_draft_task,
    report_critique_task,
)

all_tasks = [
    ("memory_retrieval_task", memory_retrieval_task),
    ("earnings_analysis_task", earnings_analysis_task),
    ("news_analysis_task", news_analysis_task),
    ("market_analysis_task", market_analysis_task),
    ("advisory_draft_task", advisory_draft_task),
    ("report_critique_task", report_critique_task),
]

for name, t in all_tasks:
    try:
        desc = (t.description or "").strip().split("\n")[0]
        ctx_raw = getattr(t, "context", None)
        ctx_len = len(ctx_raw) if isinstance(ctx_raw, (list, tuple)) else 0
        ag = getattr(t, "agent", None)
        ag_role = getattr(ag, "role", None) if hasattr(ag, "role") else None
        print(f"✓ {name}: desc='{desc[:80]}', context={ctx_len}, agent={ag_role}")
    except Exception:
        # Suppress noisy errors (e.g., NotSpecified); skip printing
        continue

##  Utility: Demo fallback when full Crew run is unavailable

In [ ]:
import requests
from datetime import datetime

def demo_fallback_snapshot(ticker: str) -> str:
    print("[Demo mode] Running simplified snapshot.\n")
    # Stock via yfinance
    price = None
    name = ticker
    sector = ""
    try:
        import yfinance as yf
        stock = yf.Ticker(ticker)
        info = getattr(stock, "info", {}) or {}
        name = info.get("longName") or name
        sector = info.get("sector") or sector
        price = info.get("regularMarketPrice")
        if price is None:
            hist = stock.history(period="5d")
            if not hist.empty:
                price = float(hist["Close"].iloc[-1])
    except Exception:
        pass

    # News via NewsAPI
    news_titles = []
    key = os.getenv("NEWSAPI_KEY") or os.getenv("NEWS_API_KEY")
    if key:
        try:
            r = requests.get(
                "https://newsapi.org/v2/everything",
                params={"q": ticker, "sortBy": "publishedAt", "pageSize": 5, "language": "en", "apiKey": key},
                timeout=20,
            )
            data = r.json() if r.ok else {}
            for a in (data.get("articles") or [])[:5]:
                title = a.get("title") or ""
                if title:
                    news_titles.append(title)
        except Exception:
            pass

    # Macro via FRED
    fred_summary = {}
    fred_key = os.getenv("FRED_API_KEY")
    if fred_key:
        try:
            from fredapi import Fred
            fred = Fred(api_key=fred_key)
            cpi = fred.get_series("CPIAUCSL").tail(1)
            gdp = fred.get_series("GDP").tail(1)
            if not cpi.empty:
                fred_summary["CPIAUCSL"] = float(cpi.iloc[0])
            if not gdp.empty:
                fred_summary["GDP"] = float(gdp.iloc[0])
        except Exception:
            pass

    lines = [
        f"Demo Investment Snapshot ({datetime.now().strftime('%Y-%m-%d %H:%M')})",
        f"Ticker: {ticker}",
        f"Company: {name}",
        f"Sector: {sector}" if sector else "Sector: n/a",
        f"Last Price: {price}" if price is not None else "Last Price: n/a",
    ]
    if news_titles:
        lines.append("Top News: " + " | ".join(news_titles[:3]))
    if fred_summary:
        lines.append("Macro: " + ", ".join(f"{k}={v}" for k, v in fred_summary.items()))

    report = "\n".join(lines)
    print(report)
    return report

## Validate environment and run orchestration

In [ ]:
import traceback

# Accept NEWS_API_KEY alias
if "NEWSAPI_KEY" not in os.environ and os.getenv("NEWS_API_KEY"):
    os.environ["NEWSAPI_KEY"] = os.getenv("NEWS_API_KEY")

required_keys = ["OPENAI_API_KEY", "FRED_API_KEY"]
missing = [k for k in required_keys if not os.getenv(k)]
if missing:
    print("Missing required keys:", missing)
    print("Add them to .env and rerun. Falling back to demo if FORCE_DEMO=1 or OPENAI key missing.")

use_demo = os.getenv("FORCE_DEMO", "0") == "1" or ("OPENAI_API_KEY" in missing)

ticker = os.getenv("DEMO_TICKER", "MSFT")

if use_demo:
    _ = demo_fallback_snapshot(ticker)
else:
    try:
        # Orchestrated CrewAI run
        from orchestrator import run_analysis
        print(f"Starting multi-agent analysis for {ticker}...")
        run_analysis(ticker)
    except Exception as e:
        print("[ERROR] Orchestrator run failed:", e)
        print("Falling back to demo snapshot...\n")
        try:
            _ = demo_fallback_snapshot(ticker)
        except Exception as e2:
            print("[ERROR] Demo fallback also failed:", e2)
            traceback.print_exc()